In [3]:
import os
import sys
import pandas as pd
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [130]:
df = pd.read_csv('./data/job_opening.csv')

In [131]:
documents = [Document(page_content=f"{row['rec_idx']}: {row['jd_text']} {row['recruit_title']} {row['recruit_kewdcdnm']} {row['company_place']} {row['career']} {row['education']}", metadata={"id": idx}) for idx, row in df.iterrows()]

In [117]:
# chromadb 연결

client = chromadb.HttpClient(host='43.202.186.183', port=8000, settings=Settings(allow_reset=True, anonymized_telemetry=False))

In [132]:
# openai 임베딩 생성
from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions

load_dotenv()

embeddings = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ.get('OPENAI_API_KEY'),
        model_name="text-embedding-ada-002"
    )


In [136]:
#collection 생성

from datetime import datetime

collection = client.create_collection(
    name="job_opening", 
    embedding_function=embeddings,
    metadata={
        "description": "for recommend job_opening",
        "created": str(datetime.now())
    }  
)


In [137]:
collection = client.get_collection(name="job_opening")

In [46]:
documents[0].metadata['id']

0

In [138]:
# 문서 삽입
try:
    for document in documents:
        if document.page_content is not None:
            collection.add(
                documents = [document.page_content],
                ids = [str(document.metadata['id'])]
            )
except Exception as e:
    print(f"An error occurred: {e}")

In [139]:
db = Chroma(client=client, collection_name="job_opening",embedding_function=embeddings)

In [140]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 20}
)

docs = retriever.invoke("데이터 분석 AI 모델링 감성 분석 영화 추천 시스템 개발 IT 빅데이터와 관련된 문서를 찾고싶어.")

AttributeError: 'OpenAIEmbeddingFunction' object has no attribute 'embed_query'

In [101]:
docs

[Document(metadata={}, page_content="공고ID: 50139999, 공고내용: \n채용공고 상세\n[코스닥기업] 업무시스템 개발자 채용2005년 04월 02일에 설립된 응용 소프트웨어 개발 및 공급업업종의 AI기반데이터 분석,로우코드솔루션 개발,공급사업을 하는 코스닥,중소기업,주식회사기업 입니다.모집부문 및 상세내용모집부문상세내용업무시스템 개발자2명ㆍ솔루션 기반 업무시스템 개발ㆍ솔루션 개발 리더자격요건주요업무담당업무ㆍ자격요건ㆍ관련 경력 5년 이상ㆍ DB, SQL 및 데이터 지식 보유자ㆍjavascript 사용 가능자우대사항ㆍ동종업계 관련 업무 경험자ㆍ솔루션을 활용한 포털 및 업무시스템 구축 경험자ㆍ고객사 사용자 관점의 우수한 커뮤니케이션 스킬 보유자\nㆍ기타 필수 사항\n우대사항근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:ㆍ근무지역:(135-080) 서울 강남구 역삼동 708-8 세방빌딩 신관17층 -\xa0서울 2호선 선릉역에서 1분 이내전형절차 서류전형(※서류합격자에 한하여간단한 전화인터뷰 진행) 1차면접 2차면접 최종합격접수기간 및 방법ㆍ접수기간:2025년 3월 4일 (화) 17시~ 2025년 3월 31일 (월) 24시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:이력서\xa0및 자기소개서(사진, 연락처, 희망연봉 반드시 기재요망)경력기술서 (프로젝트별 Skill inventory 포함 우대)ㆍ제출서류:유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상이할 수 있으니 유의하시길 바랍니다.\n, 공고제목: [코스닥기업] 업무시스템 개발자 채용, 직무구분: ['웹개발', '데이터시각화', '솔루션', '컨설턴트', 'BI 엔지니어'], 지역: 서울 강남구, 경력구분: 경력 5년↑ · 정규직, 학력: 대학(2,3년)↑"),
 Document(metadata={}, page_content="공고ID: 50283013

In [37]:
rec_ids = [ doc.page_content.split(', ')[0].split(': ')[1] for doc in docs ] 

In [38]:
rec_ids

['50139999',
 '50197577',
 '50283013',
 '50235054',
 '50282974',
 '50179159',
 '50213940',
 '50235424',
 '50102550',
 '50234809',
 '50222180',
 '50283469',
 '50042760',
 '50283455',
 '50235644',
 '50111469',
 '50215717',
 '50254512',
 '50207599',
 '50274739']

In [73]:
from langchain_core.runnables import RunnableMap
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda

llm = ChatOpenAI(
        model="gpt-4",
        temperature=0
    )

In [74]:
prompt = PromptTemplate.from_template(
    """
    당신은 채용 공고 추천 전문가입니다.
    참고 문서:
    {docs}

    질문:
    {question}
    """

    )

# chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [75]:
# 체인 생성
chain = LLMChain(
    llm=llm, 
    prompt=prompt)

retriever_chain = RunnableLambda(lambda x: {"docs": retriever.get_relevant_documents(x["question"])})

full_chain = RunnableMap({
    "question": lambda x: x["question"],
    "docs": lambda x: retriever.get_relevant_documents(x["question"])
}) | chain

# # invoke 호출
# response = chain.invoke({
#     "docs": docs
# })
# result = response['text']

response = full_chain.invoke({"question": """
    사용자 정보는 다음과 같습니다:
    - 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발
    - 관심 산업/도메인: IT, 빅데이터
    - 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit
    - 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입
    - 희망 지역: 서울특별시 강남구

    ### 출력 형식 예시
    1. **직무명**: AI 분석가 (신입)
    - **산업 분야**: IT 서비스 / 빅데이터 분석
    - **필요 기술**: Python, TensorFlow, SQL, Pandas
    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999
    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.

    사용자 정보에 가장 적합한 채용 공고 20개를 추천해주세요.
    출력 형식과 동일한 형식으로 출력해주세요.
"""})

print(response)

{'question': '\n    사용자 정보는 다음과 같습니다:\n    - 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발\n    - 관심 산업/도메인: IT, 빅데이터\n    - 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit\n    - 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입\n    - 희망 지역: 서울특별시 강남구\n\n    ### 출력 형식 예시\n    1. **직무명**: AI 분석가 (신입)\n    - **산업 분야**: IT 서비스 / 빅데이터 분석\n    - **필요 기술**: Python, TensorFlow, SQL, Pandas\n    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999\n    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.\n\n    사용자 정보에 가장 적합한 채용 공고 20개를 추천해주세요.\n    출력 형식과 동일한 형식으로 출력해주세요.\n', 'docs': [Document(metadata={}, page_content="공고ID: 50197577, 공고내용: \n채용공고 상세\nㅡ\n㈜경동도시가스\n 데이터 분석 및 관련\n사업개발 전문가 채용\n데이터 분석 및 관련사업 전문가 (정규직)\n\xa0\n담당업무 \n\nㆍ데이터 관련 시장동향 분석\nㆍ신규 사업기회 발굴 및 \nㆍ관련 투자사 사업지원\nㆍ빅데이터, AI분

In [76]:
response

{'question': '\n    사용자 정보는 다음과 같습니다:\n    - 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발\n    - 관심 산업/도메인: IT, 빅데이터\n    - 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit\n    - 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입\n    - 희망 지역: 서울특별시 강남구\n\n    ### 출력 형식 예시\n    1. **직무명**: AI 분석가 (신입)\n    - **산업 분야**: IT 서비스 / 빅데이터 분석\n    - **필요 기술**: Python, TensorFlow, SQL, Pandas\n    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999\n    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.\n\n    사용자 정보에 가장 적합한 채용 공고 20개를 추천해주세요.\n    출력 형식과 동일한 형식으로 출력해주세요.\n',
 'docs': [Document(metadata={}, page_content="공고ID: 50197577, 공고내용: \n채용공고 상세\nㅡ\n㈜경동도시가스\n 데이터 분석 및 관련\n사업개발 전문가 채용\n데이터 분석 및 관련사업 전문가 (정규직)\n\xa0\n담당업무 \n\nㆍ데이터 관련 시장동향 분석\nㆍ신규 사업기회 발굴 및 \nㆍ관련 투자사 사업지원\nㆍ빅데이터, AI

In [88]:
response['docs'][0].page_content.split(", ")[0].split(": ")[1]

'50197577'

In [91]:
rec_idx = [ res.page_content.split(", ")[0].split(": ")[1] for res in response['docs'] ] 

In [92]:
rec_idx

['50197577', '50139999', '50235054', '50102550']

In [7]:
chroma_client = chromadb.HttpClient(
    host="43.202.186.183",
    port=8000,
    settings=Settings(allow_reset=True, anonymized_telemetry=False)
)
collection = chroma_client.get_collection(name="job_position")

results = collection.get()
print("총 문서 수:", len(results['documents']))
print("예시 문서:", results['documents'][0])

총 문서 수: 13853
예시 문서: 공고내용: [3500/중식제공]식품 쇼핑몰 MD 경력직 채용

채용공고 상세, 회사명: (주)다온미트, 직무: ['온라인MD'], 지역: 경기 하남시, 경력: 경력(년수무관) · 정규직, 학력: 학력무관
